In [ ]:
import torch
from torch import nn

import fastai
from fastai.vision.all import *
from torchvision.models import ResNet50_Weights

from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

# fastai.vision.data.device = torch.device("cpu")

In [ ]:
print(torch.cuda.is_available())

In [ ]:
class_names = np.load("class_names.npy", allow_pickle=True)
# print(class_names.item())

attributes = np.load("attributes.npy", allow_pickle=True)
attributes.shape

In [ ]:
train_df = pd.read_csv("train_images.csv")
test_df = pd.read_csv("test_images_path.csv")

dls = ImageDataLoaders.from_df(
    train_df,
    valid_pct=0.2,
    item_tfms=Resize(224),
    bs=64,
    label_col="label",
    fn_col="image_path",
)

In [ ]:
dls.train.item_tfms = [
    RandomResizedCrop(224, min_scale=0.5),
    Rotate(),
    Flip(),
    Dihedral(),
    Brightness(),
    Contrast(),
    Saturation(),
    Hue(),
]

## Modelling


In [ ]:
learn = vision_learner(dls, resnet34, metrics=accuracy)

In [ ]:
learn.lr_find()

In [ ]:
learn.fine_tune(4)

In [ ]:
learn.show_results()

## Testing


In [ ]:
test_dl = dls.test_dl(test_df, with_labels=True)
predictions, targets = learn.get_preds(dl=test_dl)

In [ ]:
acc = accuracy(predictions, targets)
print(f"Accuracy: {acc.item() * 100:.2f}%")

## Creating Sample


In [ ]:
# Create a submission DataFrame
submission_df = test_df.copy()
submission_df["label"] = predicted_labels

# Optional: if your test_df contains a column that represents an ID, use that instead
# For example, if there's an 'image_id' column:
# submission_df = pd.DataFrame({"id": test_df["image_id"], "label": predicted_labels})

# Save the submission file
submission_df[["id", "label"]].to_csv("submission.csv", index=False)

In [ ]:
torch.cuda.empty_cache()